In [ ]:
# Pooled Cumulative Incidence Curve – Optum Dataset

# This notebook generates a cumulative incidence curve for a cohort used in the tirzepatide vs dulaglutide for major adverse cardiovascular events.
# It plots cumulative incidence of the composite outcome.

# Required Inputs
## `pooled_dataset.csv` containing:
## `time`: follow-up time
## `event`: binary outcome indicator
## `treatment_group`: exposure (e.g. tirepatide vs dulaglutide)

In [1]:
import pandas as pd
import numpy as np
import scipy
import lifelines
import pandas
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

# Load datasets
merged_optum_data = pd.read_csv("path/to/cohort_file.csv")
merged_optum_data_key = pd.read_csv("path/to/cohort_file.csv")

# Create and apply a column renaming mapping for merged_optum_data
cohort_mapping = dict(zip(merged_optum_data_key["Column Name"], merged_optum_data_key["Description"]))
cohort_mapping.pop('PID', None)  # Ensure 'PID' remains unchanged
trimmed_cohort_mapping = {col: desc.split(" - ")[-1] for col, desc in cohort_mapping.items()}
merged_optum_data.rename(columns=trimmed_cohort_mapping, inplace=True)

# Inspect the final dataset
merged_optum_data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from matplotlib import font_manager as fm

# Step 1: Prepare data
merged_optum_data["Event"] = merged_optum_data["MACE"].astype(int)
merged_optum_data["Follow Up Time"] = np.minimum(merged_optum_data["Follow Up Time"], 365)
merged_optum_data["Exposure Binary"] = merged_optum_data["Exposure Group"].map({"E": 1, "R": 0})

# Step 2: Fit KM curves
km_tirz = KaplanMeierFitter()
km_dula = KaplanMeierFitter()

km_tirz.fit(
    durations=merged_optum_data.loc[merged_optum_data["Exposure Binary"] == 1, "Follow Up Time"],
    event_observed=merged_optum_data.loc[merged_optum_data["Exposure Binary"] == 1, "Event"],
    label="Tirzepatide"
)

km_dula.fit(
    durations=merged_optum_data.loc[merged_optum_data["Exposure Binary"] == 0, "Follow Up Time"],
    event_observed=merged_optum_data.loc[merged_optum_data["Exposure Binary"] == 0, "Event"],
    label="Dulaglutide"
)

# Define colors
tirz_red = "#a42b44"      # Tirzepatide color from image
dula_gray = "#999999"     # Dulaglutide line
dula_gray_ci = "#aaaaaa"  # Dulaglutide CI band

fig, ax = plt.subplots(figsize=(6, 6))

# Tirzepatide: use lifelines' built-in plot
km_tirz.plot_cumulative_density(ax=ax, color=tirz_red, label="Tirzepatide", ci_show=True, lw=2)

# Dulaglutide: plot curve and CI manually
dula_curve = km_dula.cumulative_density_[km_dula.label]
ax.plot(
    dula_curve.index,
    dula_curve,
    label=km_dula.label,
    color=dula_gray,
    lw=2
)

# CI band for Dulaglutide
ci_lower = km_dula.confidence_interval_cumulative_density_[f"{km_dula.label}_lower_0.95"]
ci_upper = km_dula.confidence_interval_cumulative_density_[f"{km_dula.label}_upper_0.95"]
ax.fill_between(
    km_dula.cumulative_density_.index,
    ci_lower,
    ci_upper,
    color=dula_gray_ci,
    alpha=0.25
)

# X-axis (months)
month_days = np.array([0, 3, 6, 9, 12]) * 30
month_days[-1] = 365
ax.set_xlim(0, 365)
ax.set_xticks(month_days)
ax.set_xticklabels([f"{m}" for m in [0, 3, 6, 9, 12]])

# Y-axis (percent scale with grid)
ax.set_ylim(0.00, 0.10)
ax.set_yticks(np.arange(0.00, 0.11, 0.02))
ax.set_yticklabels([f"{int(y*100)}" for y in np.arange(0.00, 0.11, 0.02)])

# Axis labels
ax.set_xlabel("Follow-up, mo")
ax.set_ylabel("Cumulative incidence, %")

# Legend
legend = ax.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98), frameon=True,
                   handlelength=2, handletextpad=1, borderpad=0.2, labelspacing=0.4)
legend.get_frame().set_boxstyle("square")
legend.get_frame().set_linewidth(1.0)

# Grid and layout
ax.grid(True, which='major', axis='y', linestyle='-', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Extract point estimates
# Tirzepatide
tirz_surv = km_tirz.predict(365)
tirz_ci = km_tirz.confidence_interval_.loc[365]
tirz_label = km_tirz.label
tirz_lower_surv = tirz_ci[f"{tirz_label}_lower_0.95"]
tirz_upper_surv = tirz_ci[f"{tirz_label}_upper_0.95"]
tirz_cum = 1 - tirz_surv
tirz_lower = 1 - tirz_upper_surv
tirz_upper = 1 - tirz_lower_surv

# Dulaglutide
dula_surv = km_dula.predict(365)
dula_ci = km_dula.confidence_interval_.loc[365]
dula_label = km_dula.label
dula_lower_surv = dula_ci[f"{dula_label}_lower_0.95"]
dula_upper_surv = dula_ci[f"{dula_label}_upper_0.95"]
dula_cum = 1 - dula_surv
dula_lower = 1 - dula_upper_surv
dula_upper = 1 - dula_lower_surv

# Calculate ARD (using MACE as outcome)
def bootstrap_ard(df, n_iterations=1000, seed=42):
    np.random.seed(seed)
    boot_ard = []

    for _ in range(n_iterations):
        boot_df = df.sample(n=len(df), replace=True)

        # Ensure follow-up is capped and binary event is clean
        boot_df["Follow Up Time"] = np.minimum(boot_df["Follow Up Time"], 365)
        boot_df["Event"] = boot_df["MACE"].astype(int)
        boot_df["Exposure Binary"] = boot_df["Exposure Group"].map({"E": 1, "R": 0})

        km = KaplanMeierFitter()
        results = {}

        for group_name, group_val in [("Tirzepatide", 1), ("Dulaglutide", 0)]:
            mask = boot_df["Exposure Binary"] == group_val
            km.fit(
                durations=boot_df.loc[mask, "Follow Up Time"],
                event_observed=boot_df.loc[mask, "Event"]
            )
            surv_prob = km.predict(365)
            results[group_name] = 1 - surv_prob

        boot_ard.append(results["Tirzepatide"] - results["Dulaglutide"])

    return np.array(boot_ard)

boot_ard_dist = bootstrap_ard(merged_optum_data, n_iterations=1000)
ard_point = tirz_cum - dula_cum
ard_lower = np.percentile(boot_ard_dist, 2.5)
ard_upper = np.percentile(boot_ard_dist, 97.5)

# Create summary DataFrame with fixed 2-decimal formatting
summary_df = pd.DataFrame({
    "Comparison": [
        f"{tirz_label}",
        f"{dula_label}",
        f"{tirz_label} vs {dula_label} (ARD)"
    ],
    "Estimate (%)": [
        f"{tirz_cum * 100:.2f}",
        f"{dula_cum * 100:.2f}",
        f"{ard_point * 100:.2f}"
    ],
    "95% CI (%)": [
        f"{tirz_lower * 100:.2f}–{tirz_upper * 100:.2f}",
        f"{dula_lower * 100:.2f}–{dula_upper * 100:.2f}",
        f"{ard_lower * 100:.2f}–{ard_upper * 100:.2f}"
    ]
})

summary_df